In [231]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from torch_geometric.utils import softmax
import torch_geometric.utils.convert as conv
import torch
from torch_geometric.nn import GCNConv, GATConv,SAGEConv
import torch.nn.functional as F
from torch.nn.functional import softmax

# load the graph from the graphml file
def load_graph(file_path):
    G = nx.read_graphml(file_path)
    return G

file_path = 'data/cora.graphml'
# define the classification label
classification_label = 'subject'


In [69]:
G = load_graph(file_path)

In [70]:
#Removing not useful attributes
for id in G.nodes:
    del G.nodes[id]['label']
for s,d in G.edges:
    del G.edges[s,d]['label']

In [71]:

print(G, type(G))

print(list(G.nodes(data=True))[0])


Graph with 2708 nodes and 5278 edges <class 'networkx.classes.graph.Graph'>
('1033', {'w_0': 0, 'w_1': 0, 'w_2': 0, 'w_3': 0, 'w_4': 0, 'w_5': 0, 'w_6': 0, 'w_7': 0, 'w_8': 0, 'w_9': 1, 'w_10': 0, 'w_11': 0, 'w_12': 0, 'w_13': 0, 'w_14': 0, 'w_15': 0, 'w_16': 0, 'w_17': 0, 'w_18': 0, 'w_19': 0, 'w_20': 0, 'w_21': 0, 'w_22': 0, 'w_23': 0, 'w_24': 0, 'w_25': 0, 'w_26': 0, 'w_27': 0, 'w_28': 0, 'w_29': 0, 'w_30': 0, 'w_31': 0, 'w_32': 0, 'w_33': 0, 'w_34': 0, 'w_35': 0, 'w_36': 0, 'w_37': 0, 'w_38': 0, 'w_39': 0, 'w_40': 0, 'w_41': 0, 'w_42': 0, 'w_43': 0, 'w_44': 0, 'w_45': 0, 'w_46': 0, 'w_47': 0, 'w_48': 0, 'w_49': 0, 'w_50': 0, 'w_51': 0, 'w_52': 0, 'w_53': 0, 'w_54': 0, 'w_55': 0, 'w_56': 0, 'w_57': 0, 'w_58': 0, 'w_59': 0, 'w_60': 0, 'w_61': 0, 'w_62': 0, 'w_63': 0, 'w_64': 0, 'w_65': 0, 'w_66': 0, 'w_67': 0, 'w_68': 0, 'w_69': 0, 'w_70': 0, 'w_71': 0, 'w_72': 0, 'w_73': 0, 'w_74': 0, 'w_75': 0, 'w_76': 0, 'w_77': 0, 'w_78': 0, 'w_79': 0, 'w_80': 0, 'w_81': 0, 'w_82': 0, 'w_83': 0, 

In [72]:
edge = list(G.edges(data=True))[0]

print(edge)


('1033', '35', {})


Convert the node attributes into integers

In [102]:
classes = set()
for id in G.nodes:
    classes.add(G.nodes[id]['subject'])

labels = {'Probabilistic_Methods':0,
          'Theory':1,
          'Reinforcement_Learning':2,
          'Case_Based':3,
          'Genetic_Algorithms':4,
          'Neural_Networks':5,
          'Rule_Learning':6,
          0:0,1:1,2:2,3:3,4:4,5:5,6:6}
for id in G.nodes:
    G.nodes[id]['subject'] = labels[G.nodes[id]['subject']]

g_node_attrs = list(list(G.nodes(data=True))[0][1].keys())

print(g_node_attrs)



num_classes = len(classes)

print(classes)

w_attrs = [attr for attr in g_node_attrs if 'w_' in attr]

print(len(g_node_attrs))

['w_0', 'w_1', 'w_2', 'w_3', 'w_4', 'w_5', 'w_6', 'w_7', 'w_8', 'w_9', 'w_10', 'w_11', 'w_12', 'w_13', 'w_14', 'w_15', 'w_16', 'w_17', 'w_18', 'w_19', 'w_20', 'w_21', 'w_22', 'w_23', 'w_24', 'w_25', 'w_26', 'w_27', 'w_28', 'w_29', 'w_30', 'w_31', 'w_32', 'w_33', 'w_34', 'w_35', 'w_36', 'w_37', 'w_38', 'w_39', 'w_40', 'w_41', 'w_42', 'w_43', 'w_44', 'w_45', 'w_46', 'w_47', 'w_48', 'w_49', 'w_50', 'w_51', 'w_52', 'w_53', 'w_54', 'w_55', 'w_56', 'w_57', 'w_58', 'w_59', 'w_60', 'w_61', 'w_62', 'w_63', 'w_64', 'w_65', 'w_66', 'w_67', 'w_68', 'w_69', 'w_70', 'w_71', 'w_72', 'w_73', 'w_74', 'w_75', 'w_76', 'w_77', 'w_78', 'w_79', 'w_80', 'w_81', 'w_82', 'w_83', 'w_84', 'w_85', 'w_86', 'w_87', 'w_88', 'w_89', 'w_90', 'w_91', 'w_92', 'w_93', 'w_94', 'w_95', 'w_96', 'w_97', 'w_98', 'w_99', 'w_100', 'w_101', 'w_102', 'w_103', 'w_104', 'w_105', 'w_106', 'w_107', 'w_108', 'w_109', 'w_110', 'w_111', 'w_112', 'w_113', 'w_114', 'w_115', 'w_116', 'w_117', 'w_118', 'w_119', 'w_120', 'w_121', 'w_122', 'w

In [95]:
# delete all the graph attributes, since they are not useful for node classification

# print(type(node_attrs))
# print(edge_attrs)

# graph.graph['node_default'] = True
# graph.graph['edge_default'] = True

print(G.graph, type(G.graph))

if 'node_default' in G.graph:
    del G.graph['node_default']
if 'edge_default' in G.graph:
    del G.graph['edge_default']

if 'edge_index' in G.graph:
    del G.graph['edge_index']

attrs = [attr for attr, _ in G.graph.items()]

for attr in attrs:
    del G.graph[attr]

print(G.graph, type(G.graph))

{} <class 'dict'>
{} <class 'dict'>
0


In [249]:
data = conv.from_networkx(G, group_node_attrs=g_node_attrs)

print(f'Number of edges: {data.num_edges}')

print(data, type(data))

Number of edges: 10556
Data(edge_index=[2, 10556], x=[2708, 1434]) <class 'torch_geometric.data.data.Data'>


Extract the classification label from the nodes attributes

In [250]:
print(data.x)

# shuffle the data by row
data.x = data.x[torch.randperm(data.x.size()[0])]
print(data.x.shape)

# get the classification attibute from x and assign it to y
data.y = data.x[:, -1]
# data.y = torch.stack([y[0] for y in data.y])

# remove the classification attribute from x
data.x = data.x[:, :-1].float()

print(data.x)
print(data.y)

print(data)

tensor([[0, 0, 0,  ..., 0, 0, 4],
        [0, 0, 0,  ..., 0, 0, 4],
        [0, 0, 0,  ..., 0, 0, 5],
        ...,
        [0, 0, 0,  ..., 0, 0, 5],
        [0, 0, 0,  ..., 0, 0, 5],
        [0, 0, 0,  ..., 0, 0, 5]])
torch.Size([2708, 1434])
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
tensor([5, 0, 1,  ..., 1, 5, 3])
Data(edge_index=[2, 10556], x=[2708, 1433], y=[2708])


Define training, validation and test sets randomly

In [251]:
from torch_geometric.transforms import RandomNodeSplit

# split the data in train, validation and test sets
# it adds to data train, val and test masks
rns = RandomNodeSplit('random', num_val=0.1, num_test=0.2)

data = rns(data)

print(data)

Data(edge_index=[2, 10556], x=[2708, 1433], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])


One-hot encode the classification labels

In [252]:
# one hot encode labels
data.y = torch.nn.functional.one_hot(data.y.long(), num_classes=num_classes).to(torch.float)

print(data.y)
print(data)

tensor([[0., 0., 0.,  ..., 0., 1., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 1., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
Data(edge_index=[2, 10556], x=[2708, 1433], y=[2708, 7], train_mask=[2708], val_mask=[2708], test_mask=[2708])


In [253]:
print(data.x.dtype)
print(data.y.dtype)
print(data.edge_index.dtype)

torch.float32
torch.float32
torch.int64


In [254]:
print(list(data.edge_index))

[tensor([   0,    0,    0,  ..., 2706, 2707, 2707]), tensor([   1,  324,  330,  ..., 2705,  729, 2705])]


In [255]:
from sklearn.manifold import TSNE

def visualize(h, color):
    z = TSNE(n_components=2).fit_transform(h.detach().cpu().numpy())

    plt.figure(figsize=(10,10))
    plt.xticks([])
    plt.yticks([])

    plt.scatter(z[:, 0], z[:, 1], s=70, c=color, cmap="Set2")
    plt.show()

In [270]:
class GCN(torch.nn.Module):
    def __init__(self, input_features, hidden_channels):
        super().__init__()
        torch.manual_seed(1234567)
        self.conv1 = GCNConv(input_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, num_classes)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.2, training=self.training)
        # x = softmax(x)
        # x = F.log_softmax(x, dim=1)
        x = F.softmax(x, dim=1)
        return x

in_features = data.x.shape[1]

model = GCN(input_features=in_features, hidden_channels=16)
model.eval()
print(model)

out = model(data.x, data.edge_index)
#visualize(out, color=data.y)
print(out)

model = GCN(input_features=in_features, hidden_channels=64)
optimizer = torch.optim.Adam(model.parameters(), lr=0.1, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()


GCN(
  (conv1): GCNConv(1433, 16)
  (conv2): GCNConv(16, 7)
)
tensor([[0.1439, 0.1411, 0.1427,  ..., 0.1414, 0.1482, 0.1417],
        [0.1649, 0.1386, 0.1328,  ..., 0.1318, 0.1688, 0.1315],
        [0.1472, 0.1418, 0.1401,  ..., 0.1413, 0.1499, 0.1408],
        ...,
        [0.1505, 0.1378, 0.1378,  ..., 0.1405, 0.1562, 0.1378],
        [0.1500, 0.1395, 0.1395,  ..., 0.1395, 0.1514, 0.1395],
        [0.1517, 0.1400, 0.1354,  ..., 0.1426, 0.1559, 0.1354]],
       grad_fn=<SoftmaxBackward0>)


In [267]:
class SAGE(torch.nn.Module):
    def __init__(self, input_features, hidden_channels):
        super().__init__()
        torch.manual_seed(1234567)
        self.conv1 = SAGEConv(input_features, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, num_classes)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.2, training=self.training)
        x = F.softmax(x, dim=1)
        return x

in_features = data.x.shape[1]

model = SAGE(input_features=in_features, hidden_channels=64)
optimizer = torch.optim.Adam(model.parameters(), lr=0.1, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()


In [258]:
class GAT(torch.nn.Module):
    def __init__(self, hidden_channels, heads):
        super().__init__()
        torch.manual_seed(1234567)

        self.hid = 8
        self.in_head = 8
        self.out_head = 1

        self.conv1 = GATConv(in_features, self.hid, heads=self.in_head, dropout=0.6)
        self.conv2 = GATConv(self.hid*self.in_head, num_classes, concat=False,
                             heads=self.out_head, dropout=0.6)


    def forward(self, x, edge_index):

        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv1(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, edge_index)

        return F.softmax(x, dim=1)

model = GAT(hidden_channels=8, heads=8)
print(model)

optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()

GAT(
  (conv1): GATConv(1433, 8, heads=8)
  (conv2): GATConv(64, 7, heads=1)
)


In [271]:
def train():
      model.train()
      optimizer.zero_grad()  # Clear gradients.
      out = model(data.x, data.edge_index)  # Perform a single forward pass.
      loss = criterion(out[data.train_mask], data.y[data.train_mask])
      val_loss = criterion(out[data.val_mask], data.y[data.val_mask])  # Compute the loss solely based on the training nodes.
      loss.backward()  # Derive gradients.
      optimizer.step()  # Update parameters based on gradients.
      return loss, val_loss


def test(models, ds='test'):
      models.eval()
      out = models(data.x, data.edge_index)
      pred = out.argmax(dim=1)  # Use the class with highest probability.
      if ds == 'test':
            outs = out[data.test_mask]
            mask = data.test_mask
            d_set = pred[data.test_mask]
            label_set = data.y[data.test_mask]
      elif ds == 'val':
            outs = out[data.val_mask]
            mask = data.val_mask
            d_set = pred[data.val_mask]
            label_set = data.y[data.val_mask]
      else:
            outs = out[data.train_mask]
            mask = data.train_mask
            d_set = pred[data.train_mask]
            label_set = data.y[data.train_mask]
      print(outs)
      print(label_set)
      label_set = label_set.argmax(dim=1)
      test_correct = d_set == label_set  # Check against ground-truth labels.
      test_acc = int(test_correct.sum()) / int(mask.sum())  # Derive ratio of correct predictions.
      return test_acc

In [272]:
patience = 7
best_val_loss = 10000
best_model = model

for epoch in range(1, 101):
    if patience == 0:
        break
    loss, val_loss = train()
    if val_loss <= best_val_loss:
        patience = 10
        best_val_loss = val_loss
        best_model = model
    else:
        patience -= 1

    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Val Loss: {val_loss:.4f}')


train_acc = test(best_model, 'train')
val_acc = test(best_model, 'val')
test_acc = test(best_model)

print(f'Train Accuracy: {train_acc:.4f}')
print(f'Val Accuracy: {val_acc:.4f}')
print(f'Test Accuracy: {test_acc:.4f}')

model.eval()

# out = model(data.x, data.edge_index)
# visualize(out, color=data.y)

Epoch: 001, Loss: 1.9460, Val Loss: 1.9456
Epoch: 002, Loss: 1.9290, Val Loss: 1.9762
Epoch: 003, Loss: 1.8945, Val Loss: 1.9893
Epoch: 004, Loss: 1.8162, Val Loss: 1.9501
Epoch: 005, Loss: 1.8269, Val Loss: 1.9155
Epoch: 006, Loss: 1.7592, Val Loss: 1.9117
Epoch: 007, Loss: 1.7146, Val Loss: 1.9636
Epoch: 008, Loss: 1.6222, Val Loss: 1.9073
Epoch: 009, Loss: 1.6570, Val Loss: 1.9672
Epoch: 010, Loss: 1.5744, Val Loss: 1.9602
Epoch: 011, Loss: 1.5380, Val Loss: 1.8775
Epoch: 012, Loss: 1.5186, Val Loss: 1.8674
Epoch: 013, Loss: 1.4628, Val Loss: 1.9254
Epoch: 014, Loss: 1.4287, Val Loss: 1.9179
Epoch: 015, Loss: 1.4827, Val Loss: 1.9004
Epoch: 016, Loss: 1.4091, Val Loss: 1.8846
Epoch: 017, Loss: 1.4969, Val Loss: 1.8544
Epoch: 018, Loss: 1.4423, Val Loss: 1.8878
Epoch: 019, Loss: 1.4261, Val Loss: 1.8937
Epoch: 020, Loss: 1.4522, Val Loss: 1.8653
Epoch: 021, Loss: 1.3979, Val Loss: 1.8792
Epoch: 022, Loss: 1.3536, Val Loss: 1.8772
Epoch: 023, Loss: 1.3994, Val Loss: 1.9012
Epoch: 024,

GCN(
  (conv1): GCNConv(1433, 64)
  (conv2): GCNConv(64, 7)
)